# Processing data for modelling

## - Stock price

In [ ]:
import pandas as pd
from tqdm import tqdm

from functions import print_shape


In [ ]:
# test with feature_engineering.py
prices = pd.read_csv('data/train_files/stock_prices.csv', parse_dates=[1])
print_shape(prices)
prices.head(2)

In [ ]:

from feature_engineering import fill_and_drop_na_values

fill_prices = fill_and_drop_na_values(prices)
print_shape(fill_prices)
fill_prices.head(2)


In [ ]:
from feature_engineering import adjust_price

ad_price = adjust_price(fill_prices)
print_shape(ad_price)
ad_price.head(2)

In [ ]:
ad_price.to_csv('data/curr_ad_price.csv') # TODO index column as index

In [1]:
import pandas as pd
from tqdm import tqdm

from functions import print_shape
ad_price = pd.read_csv('data/curr_ad_price.csv', parse_dates=['Date'])
print_shape(ad_price)

 Shape:
 ----------------------------------------
 Observations:   2.33M
 Features:       19
 Feature Date:    datetime64[ns]


In [ ]:
#from importlib import reload
#import feature_engineering as ft

from feature_engineering import price_new_features

#ft = reload(ft)
ad_price_feat = price_new_features(ad_price)
print_shape(ad_price_feat)

In [2]:
from functions import df_security_code
from feature_engineering import price_new_features

df = df_security_code(ad_price)
df_feat = price_new_features(df)
print_shape(df_feat)

  0%|          | 0/1 [00:00<?, ?it/s]DEBUG:root:Features + SMA
DEBUG:root: RSI
DEBUG:root: Return
DEBUG:root: MACD
100%|██████████| 1/1 [00:00<00:00,  8.28it/s]

 Shape:
 ----------------------------------------
 Observations:   1.2K
 Features:       36
 Feature Date:    datetime64[ns]


In [4]:
from functions import missingValues
missingValues(df_feat)

Unnamed: 0 missing Values: 0 (0.0%)
RowId missing Values: 0 (0.0%)
Date missing Values: 0 (0.0%)
SecuritiesCode missing Values: 0 (0.0%)
Open missing Values: 0 (0.0%)
High missing Values: 0 (0.0%)
Low missing Values: 0 (0.0%)
Close missing Values: 0 (0.0%)
Volume missing Values: 0 (0.0%)
AdjustmentFactor missing Values: 0 (0.0%)
ExpectedDividend missing Values: 34 (2.8%)
SupervisionFlag missing Values: 0 (0.0%)
Target missing Values: 0 (0.0%)
ad_Open missing Values: 0 (0.0%)
ad_High missing Values: 0 (0.0%)
ad_Low missing Values: 0 (0.0%)
ad_Close missing Values: 0 (0.0%)
ad_Volume missing Values: 0 (0.0%)
ad_Target missing Values: 2 (0.2%)
ad_Close_lag1 missing Values: 1 (0.1%)
ad_Close_sma10 missing Values: 9 (0.7%)
ad_Open_lag1 missing Values: 1 (0.1%)
ad_Open_sma10 missing Values: 9 (0.7%)
ad_High_lag1 missing Values: 1 (0.1%)
ad_High_sma10 missing Values: 9 (0.7%)
ad_Low_lag1 missing Values: 1 (0.1%)
ad_Low_sma10 missing Values: 9 (0.7%)
ad_Volume_lag1 missing Values: 1 (0.1%)
ad_

ad_Close_lag1 missing Values: 2332531 (100.0%)
ad_Close_sma10 missing Values: 2332531 (100.0%)
ad_Open_lag1 missing Values: 2332531 (100.0%)
ad_Open_sma10 missing Values: 2332531 (100.0%)
ad_High_lag1 missing Values: 2332531 (100.0%)
ad_High_sma10 missing Values: 2332531 (100.0%)
ad_Low_lag1 missing Values: 2332531 (100.0%)
ad_Low_sma10 missing Values: 2332531 (100.0%)
ad_Volume_lag1 missing Values: 2332531 (100.0%)
ad_Volume_sma10 missing Values: 2332531 (100.0%)

In [ ]:
missingValues(ad_price_feat)

In [ ]:
ad_price_feat.ad_Close_lag1.describe()


In [ ]:
ad_price_feat.to_csv('data/curr_ad_price_feat.csv')

In [ ]:
from feature_engineering import encode_flag

ad_price_feat['SupervisionFlag'] = encode_flag(ad_price_feat)
print_shape(ad_price)

In [ ]:
ad_price_feat.columns

## - Financials

In [ ]:
#from feature_engineering import fill_finances

financial = financial = pd.read_csv('data/train_files/financials.csv',parse_dates=[2])
financial.head(2)

In [ ]:
from feature_engineering import fill_finances_knn



filled_finances = fill_finances_knn(financial, prices)
filled_finances.head()
filled_finances.shape

In [ ]:
# ffill and drop 


def new_features_financial(filled_finances):
    sec_codes = filled_finances.SecuritiesCode.unique()

    filled_financial_feat = pd.DataFrame(columns=filled_finances.columns)

    for i in tqdm(sec_codes):
        # select a security code
        aktie = filled_finances.query('SecuritiesCode == @i')
        aktie.sort_values('Date')
        # create new features:
        aktie['margin'] = aktie['Profit'] / aktie['NetSales'] * 100
        aktie['profit_ttm'] = aktie['Profit'].shift(3) + aktie['Profit'].shift(2) + aktie['Profit'].shift(1) + aktie['Profit']
        aktie['rev_ttm'] = aktie['NetSales'].shift(3) + aktie['NetSales'].shift(2) + aktie['NetSales'].shift(1) + aktie['NetSales']
        aktie['win_quarter_growth'] = (aktie['Profit'] - aktie['Profit'].shift(1)) / aktie['Profit'].shift(1) * 100
        aktie['rev_quarter_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(1)) / aktie['NetSales'].shift(1) * 100
        aktie['win_yoy_growth'] = (aktie['Profit'] - aktie['Profit'].shift(4)) / aktie['Profit'].shift(4) * 100
        aktie['rev_yoy_growth'] = (aktie['NetSales'] - aktie['NetSales'].shift(4)) / aktie['NetSales'].shift(4) * 100
        aktie['win_ttm_growth'] = (aktie['profit_ttm'] - aktie['profit_ttm'].shift(1)) / aktie['profit_ttm'].shift(1) * 100
        aktie['rev_ttm_growth'] = (aktie['rev_ttm'] - aktie['rev_ttm'].shift(1)) / aktie['rev_ttm'].shift(1) * 100
        aktie['margin_growth'] = (aktie['margin'] - aktie['margin'].shift()) / aktie['margin'].shift() * 100
        
        # fill
        aktie = aktie.ffill()
        aktie = aktie.dropna(axis=0)

        filled_financial_feat  = pd.concat([filled_financial_feat , aktie])

        filled_financial_feat['Date'] = pd.to_datetime(filled_financial_feat['Date']) 
    
    return filled_financial_feat

In [ ]:
filled_financial_features = new_features_financial(filled_finances)

In [ ]:
"""from functions import plot_stock
code = 1301

plot_stock(filled_financial_features, code,'win_ttm_growth' )"""

In [ ]:
filled_financial_features.head(2)


In [ ]:
filled_financial_features.SecuritiesCode = filled_financial_features.SecuritiesCode.astype(int)

In [ ]:
filled_financial_features.head(2)

In [ ]:
ad_price.head(2)

## - financials + price

In [ ]:
# create key on financial : RowId
filled_financial_features['RowId'] = filled_financial_features.Date.dt.strftime('%Y%m%d').astype(str) + '_' + filled_financial_features.SecuritiesCode.astype(str)

In [ ]:
filled_financial_features.sort_values('Date').head(20)

In [ ]:
filled_financial_features.RowId.info()

In [ ]:
# concat financials and price
price_financial = pd.merge(ad_price, filled_financial_features, how='left', on='RowId', suffixes=[None, 'f_'])
#price_financial = pd.concat([ad_price, filled_financial_features], keys='RowID')

In [ ]:
print_shape(price_financial)

In [ ]:
price_financial.head()

In [ ]:
price_financial.query('RowID == "20180209_1301"')


In [ ]:
price_financial.shape

In [ ]:
from functions import missingValues

missingValues(price_financial)

In [ ]:
price_financial.ad_Volume_sma10.nunique()

In [ ]:
# ffill and drop 

def fill_drop_pro_code(df):
    sec_codes = df.SecuritiesCode.unique()

    df_fill_na = pd.DataFrame(columns=price_financial.columns)

    for i in tqdm(sec_codes):

        current = df.query('SecuritiesCode == @i')
        current.sort_values('Date')
        current = current.ffill()
        current = current.dropna(axis=0)

        df_fill_na  = pd.concat([df_fill_na , current])

    df_fill_na['Date'] = pd.to_datetime(df_fill_na['Date'])
    
    return df_fill_na

In [ ]:
price_financial_fill = fill_drop_pro_code(price_financial)

In [ ]:
# new features
#price_financial_fill.drop_duplicates(inplace=True)
price_financial_fill.shape

In [ ]:
# save dataframe to csv
price_financial_fill.to_csv('data/current.csv')